<div align="center">

# ArchiMed Images V1.4 - Separate Lung Detection

Enhanced lung segmentation with **left/right lung differentiation** and ultra-sensitive detection.

## 🫁 Key Features
- **Separate left and right lung masks** - Creates individual PNG files for each lung
- **Color-coded visualization** - Blue for left lung, red for right lung
- **Ultra-low sensitivity thresholds** (0.0001 default for maximum detection)
- **Enhanced preprocessing** (histogram equalization + gaussian blur)
- **Multiple threshold strategy** with automatic fallbacks
- **Aggressive morphological operations** for better mask cleanup

## 📁 Output Files
For each image, creates:
- `{file_id}_left_lung_mask.png` - Left lung only
- `{file_id}_right_lung_mask.png` - Right lung only  
- `{file_id}_combined_mask.png` - Both lungs combined
- `{file_id}_overlay.png` - Color-coded visualization

</div>

In [ ]:
# Configuration
CSV_FOLDER = "/home/pyuser/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"

DOWNLOAD_PATH = '/home/pyuser/data/Paradise_Test_DICOMs'
IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'

# Workflow: ArchiMed first, then local cache
USE_ARCHIMED = True  # Try ArchiMed first (recommended)
DOWNLOAD_IF_MISSING = True  # Download from ArchiMed if files not found locally

TARGET_SIZE = (518, 518)

# Ultra-High Sensitivity Control
# Lower values = more sensitive (detects smaller/fainter lung areas)
# Higher values = less sensitive (only detects clearer lung areas)
# Range: 0.0001 (ultra sensitive) to 0.5 (less sensitive)
MODEL_SENSITIVITY = 0.0001

# Enhanced Detection Options
ENABLE_HISTOGRAM_EQUALIZATION = True  # Enhance contrast before segmentation
ENABLE_GAUSSIAN_BLUR = True           # Reduce noise before segmentation
USE_MULTIPLE_THRESHOLDS = True        # Try multiple sensitivity levels
AGGRESSIVE_MORPHOLOGY = True          # More aggressive mask cleanup
ENABLE_DEBUG_OUTPUT = False           # Print segmentation debug info

LUNG_FILL_OPACITY = 0.25
LUNG_BORDER_OPACITY = 0.50

CONVERT = True
SAVE_MASKS = True


In [ ]:
# Dependencies
import pandas as pd
import os
import pydicom
import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2
import subprocess
import sys
import glob

# ArchiMed connector initialization (V1.3 working pattern)
try:
    import ArchiMedConnector.A3_Connector as A3_Conn
    a3conn = A3_Conn.A3_Connector()
    
    # Test the connection - this works in the download_archimed_images.py file
    user_info = a3conn.getUserInfos()
    print("✅ ArchiMed connector initialized and authenticated successfully")
    print(f"👤 User: {user_info}")
    archimed_available = True
    
except Exception as e:
    print(f"⚠️ ArchiMed initialization failed: {e}")
    print("📝 Note: ArchiMed may require specific configuration or credentials")
    print("🔄 Continuing with local file processing only...")
    a3conn = None
    archimed_available = False


In [ ]:
# Segmentation model setup
segmentation_model = None
model_type = None

try:
    import torchxrayvision as xrv
    import torch
    segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
    model_type = 'torchxray'
except ImportError:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torchxrayvision"])
        import torchxrayvision as xrv
        import torch
        segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
        model_type = 'torchxray'
    except:
        model_type = 'fallback'


In [ ]:
# Enhanced preprocessing and segmentation functions for separate lung detection

def enhance_image_preprocessing(image):
    """Enhanced preprocessing for better segmentation detection"""
    if len(image.shape) == 3:
        image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        image_gray = image.copy()
    
    # Histogram equalization for better contrast
    if ENABLE_HISTOGRAM_EQUALIZATION:
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        image_gray = clahe.apply(image_gray)
    
    # Gaussian blur to reduce noise
    if ENABLE_GAUSSIAN_BLUR:
        image_gray = cv2.GaussianBlur(image_gray, (3, 3), 0)
    
    return image_gray

def segment_lungs_separate(image):
    """Returns separate left and right lung masks plus combined mask"""
    if model_type == 'torchxray' and segmentation_model is not None:
        return segment_with_torchxray_separate(image)
    else:
        return segment_with_fallback_separate(image)

def segment_with_torchxray_separate(image):
    try:
        # Enhanced preprocessing
        image_preprocessed = enhance_image_preprocessing(image)
        
        image_norm = xrv.datasets.normalize(image_preprocessed, 255)
        image_norm = image_norm[None, ...]
        
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        with torch.no_grad():
            output = segmentation_model(image_tensor)
        
        # Separate left and right lung masks
        left_lung_mask = np.zeros((512, 512))
        right_lung_mask = np.zeros((512, 512))
        
        for i, target in enumerate(segmentation_model.targets):
            if target == 'Left Lung':
                left_lung_mask = output[0, i].cpu().numpy()
            elif target == 'Right Lung':
                right_lung_mask = output[0, i].cpu().numpy()
        
        # Debug output
        if ENABLE_DEBUG_OUTPUT:
            print(f"Left lung - Min: {left_lung_mask.min():.6f}, Max: {left_lung_mask.max():.6f}, Mean: {left_lung_mask.mean():.6f}")
            print(f"Right lung - Min: {right_lung_mask.min():.6f}, Max: {right_lung_mask.max():.6f}, Mean: {right_lung_mask.mean():.6f}")
        
        # Resize both masks
        left_lung_mask = cv2.resize(left_lung_mask, (image.shape[1], image.shape[0]))
        right_lung_mask = cv2.resize(right_lung_mask, (image.shape[1], image.shape[0]))
        
        def process_lung_mask(lung_mask, lung_name):
            # Multiple threshold strategy for each lung
            if USE_MULTIPLE_THRESHOLDS:
                thresholds = [MODEL_SENSITIVITY, MODEL_SENSITIVITY * 0.5, MODEL_SENSITIVITY * 0.1, 0.0001]
                best_mask = None
                best_ratio = 0
                
                for threshold in thresholds:
                    binary_mask = (lung_mask > threshold).astype(np.uint8)
                    lung_ratio = np.sum(binary_mask) / (binary_mask.shape[0] * binary_mask.shape[1])
                    
                    if ENABLE_DEBUG_OUTPUT:
                        print(f"{lung_name} threshold {threshold:.6f}: Lung ratio = {lung_ratio:.4f}")
                    
                    # Target reasonable lung area (1%-25% of image for individual lungs)
                    if 0.01 <= lung_ratio <= 0.25:
                        best_mask = binary_mask
                        best_ratio = lung_ratio
                        break
                    elif lung_ratio > 0.0005 and best_mask is None:  # Accept very small areas as fallback
                        best_mask = binary_mask
                        best_ratio = lung_ratio
                
                if best_mask is not None:
                    binary_mask = best_mask
                    if ENABLE_DEBUG_OUTPUT:
                        print(f"{lung_name} selected mask with ratio: {best_ratio:.4f}")
                else:
                    binary_mask = (lung_mask > 0.0001).astype(np.uint8)  # Ultra-low fallback
                    if ENABLE_DEBUG_OUTPUT:
                        print(f"{lung_name} using ultra-low threshold fallback")
            else:
                binary_mask = (lung_mask > MODEL_SENSITIVITY).astype(np.uint8)
            
            # Aggressive morphological operations
            if AGGRESSIVE_MORPHOLOGY:
                kernel_size = 15 if np.sum(binary_mask) > 500 else 20
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
                binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
                binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
                
                # Fill holes
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.fillPoly(binary_mask, contours, 1)
            else:
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
                binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
                binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
            
            return binary_mask
        
        # Process each lung mask separately
        left_binary_mask = process_lung_mask(left_lung_mask, "Left lung")
        right_binary_mask = process_lung_mask(right_lung_mask, "Right lung")
        
        # Also return combined mask for compatibility
        combined_mask = np.logical_or(left_binary_mask, right_binary_mask).astype(np.uint8)
        
        return left_binary_mask, right_binary_mask, combined_mask
        
    except Exception as e:
        if ENABLE_DEBUG_OUTPUT:
            print(f"TorchXRay segmentation failed: {e}")
        return segment_with_fallback_separate(image)

def segment_with_fallback_separate(image):
    """Fallback segmentation - returns same mask for both lungs since we can't differentiate"""
    # Enhanced fallback with same preprocessing
    image_preprocessed = enhance_image_preprocessing(image)
    
    _, otsu_mask = cv2.threshold(image_preprocessed, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # More aggressive morphology for fallback
    kernel_size = 25 if AGGRESSIVE_MORPHOLOGY else 20
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask_clean = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_OPEN, kernel)
    
    combined_mask = (mask_clean > 0).astype(np.uint8)
    
    # For fallback, we return the same mask for both lungs since we can't differentiate
    return combined_mask, combined_mask, combined_mask


In [ ]:
# Enhanced overlay function with separate lung visualization

def create_v13_overlay_separate(image, left_mask, right_mask, combined_mask, resize_crop_bounds):
    """Create overlay with separate left/right lung visualization"""
    overlay = image.copy()
    if len(overlay.shape) == 2:
        overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
    
    resize_y_min, resize_x_min, resize_y_max, resize_x_max = resize_crop_bounds
    img_height, img_width = overlay.shape[:2]
    
    # Darken areas outside the resize crop
    outside_resize_mask = np.ones((img_height, img_width), dtype=bool)
    outside_resize_mask[resize_y_min:resize_y_max, resize_x_min:resize_x_max] = False
    overlay[outside_resize_mask] = (overlay[outside_resize_mask] * 0.5).astype(np.uint8)
    
    # Separate lung visualization with different colors
    left_areas = left_mask > 0
    right_areas = right_mask > 0
    
    # Left lung = Blue, Right lung = Red
    if np.any(left_areas):
        # Blue fill for left lung
        left_fill_colored = np.zeros_like(overlay)
        left_fill_colored[left_areas] = [255, 0, 0]  # Blue
        fill_opacity = max(LUNG_FILL_OPACITY, 0.3)
        overlay[left_areas] = cv2.addWeighted(
            overlay[left_areas], 1.0 - fill_opacity, 
            left_fill_colored[left_areas], fill_opacity, 0
        )
        
        # Blue border for left lung
        left_mask_uint8 = (left_mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(left_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        border_mask = np.zeros_like(left_mask, dtype=np.uint8)
        cv2.drawContours(border_mask, contours, -1, 1, thickness=4)
        
        border_areas = border_mask > 0
        if np.any(border_areas):
            left_border_colored = np.zeros_like(overlay)
            left_border_colored[border_areas] = [255, 0, 0]  # Blue
            border_opacity = max(LUNG_BORDER_OPACITY, 0.7)
            overlay[border_areas] = cv2.addWeighted(
                overlay[border_areas], 1.0 - border_opacity, 
                left_border_colored[border_areas], border_opacity, 0
            )
    
    if np.any(right_areas):
        # Red fill for right lung
        right_fill_colored = np.zeros_like(overlay)
        right_fill_colored[right_areas] = [0, 0, 255]  # Red
        fill_opacity = max(LUNG_FILL_OPACITY, 0.3)
        overlay[right_areas] = cv2.addWeighted(
            overlay[right_areas], 1.0 - fill_opacity, 
            right_fill_colored[right_areas], fill_opacity, 0
        )
        
        # Red border for right lung
        right_mask_uint8 = (right_mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(right_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        border_mask = np.zeros_like(right_mask, dtype=np.uint8)
        cv2.drawContours(border_mask, contours, -1, 1, thickness=4)
        
        border_areas = border_mask > 0
        if np.any(border_areas):
            right_border_colored = np.zeros_like(overlay)
            right_border_colored[border_areas] = [0, 0, 255]  # Red
            border_opacity = max(LUNG_BORDER_OPACITY, 0.7)
            overlay[border_areas] = cv2.addWeighted(
                overlay[border_areas], 1.0 - border_opacity, 
                right_border_colored[border_areas], border_opacity, 0
            )
    
    # Green corner brackets for combined lung area
    def draw_corner_brackets(img, x1, y1, x2, y2, color, thickness=5, length=60):
        img_height, img_width = img.shape[:2]
        x1 = max(0, min(x1, img_width - 1))
        y1 = max(0, min(y1, img_height - 1))
        x2 = max(0, min(x2, img_width - 1))
        y2 = max(0, min(y2, img_height - 1))
        
        max_length_x = min(length, (x2 - x1) // 4)
        max_length_y = min(length, (y2 - y1) // 4)
        length = max(20, min(max_length_x, max_length_y))
        
        # Corner bracket lines
        cv2.line(img, (x1, y1), (min(x1 + length, img_width-1), y1), color, thickness)
        cv2.line(img, (x1, y1), (x1, min(y1 + length, img_height-1)), color, thickness)
        cv2.line(img, (max(x2 - length, 0), y1), (x2, y1), color, thickness)
        cv2.line(img, (x2, y1), (x2, min(y1 + length, img_height-1)), color, thickness)
        cv2.line(img, (x1, max(y2 - length, 0)), (x1, y2), color, thickness)
        cv2.line(img, (x1, y2), (min(x1 + length, img_width-1), y2), color, thickness)
        cv2.line(img, (x2, max(y2 - length, 0)), (x2, y2), color, thickness)
        cv2.line(img, (max(x2 - length, 0), y2), (x2, y2), color, thickness)
    
    # Use combined mask for brackets
    lung_coords = np.where(combined_mask > 0)
    if len(lung_coords[0]) > 0:
        actual_y_min = np.min(lung_coords[0])
        actual_y_max = np.max(lung_coords[0])  
        actual_x_min = np.min(lung_coords[1])
        actual_x_max = np.max(lung_coords[1])
        
        if (actual_x_min >= 0 and actual_y_min >= 0 and 
            actual_x_max < overlay.shape[1] and actual_y_max < overlay.shape[0] and
            actual_x_max > actual_x_min and actual_y_max > actual_y_min):
            draw_corner_brackets(overlay, actual_x_min, actual_y_min, actual_x_max, actual_y_max, (0, 255, 0), 5, 60)
    
    # Cyan rectangle for crop area
    cv2.rectangle(overlay, (resize_x_min, resize_y_min), (resize_x_max, resize_y_max), (255, 255, 0), 1)
    
    # Enhanced legend with lung differentiation
    legend_height = 140
    legend_width = min(750, img_width - 20)
    legend_y_start = img_height - legend_height - 10
    
    legend_background = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
    legend_area = overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width]
    overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width] = cv2.addWeighted(
        legend_area, 0.2, legend_background, 0.8, 0
    )
    
    text_y = legend_y_start + 20
    cv2.putText(overlay, "BLUE = Left lung | RED = Right lung", 
               (20, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 2)
    cv2.putText(overlay, "GREEN = Combined lung brackets", 
               (20, text_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)
    cv2.putText(overlay, f"CYAN = Crop {TARGET_SIZE[0]}x{TARGET_SIZE[1]}", 
               (20, text_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 0), 2)
    
    # Enhanced detection features
    features = []
    if ENABLE_HISTOGRAM_EQUALIZATION: features.append("HistEq")
    if ENABLE_GAUSSIAN_BLUR: features.append("Blur")
    if USE_MULTIPLE_THRESHOLDS: features.append("MultiThresh")
    if AGGRESSIVE_MORPHOLOGY: features.append("AggroMorph")
    features_text = "+".join(features) if features else "Basic"
    
    cv2.putText(overlay, f"Sensitivity: {MODEL_SENSITIVITY} | Enhanced: {features_text}", 
               (20, text_y + 75), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
    cv2.putText(overlay, f"Model: {model_type} | Separate L/R lung detection", 
               (20, text_y + 95), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
    
    return overlay


In [ ]:
# Modified processing function to handle separate lung masks

def process_image_with_segmentation_separate(image_array, file_id):
    """Process image with separate left/right lung segmentation"""
    try:
        left_binary_mask, right_binary_mask, combined_mask = segment_lungs_separate(image_array)
        
        # Always save masks and overlays, even for small segmented areas
        if SAVE_MASKS:
            os.makedirs(MASKS_PATH, exist_ok=True)
            
            # Save individual lung masks
            left_mask_path = os.path.join(MASKS_PATH, f"{file_id}_left_lung_mask.png")
            right_mask_path = os.path.join(MASKS_PATH, f"{file_id}_right_lung_mask.png")
            combined_mask_path = os.path.join(MASKS_PATH, f"{file_id}_combined_mask.png")
            
            left_mask_image = (left_binary_mask * 255).astype(np.uint8)
            right_mask_image = (right_binary_mask * 255).astype(np.uint8)
            combined_mask_image = (combined_mask * 255).astype(np.uint8)
            
            cv2.imwrite(left_mask_path, left_mask_image)
            cv2.imwrite(right_mask_path, right_mask_image)
            cv2.imwrite(combined_mask_path, combined_mask_image)
        
        total_pixels = combined_mask.shape[0] * combined_mask.shape[1]
        lung_pixels = np.sum(combined_mask)
        lung_ratio = lung_pixels / total_pixels
        
        # Only skip cropping for very extreme cases (but still save masks/overlays above)
        if lung_ratio < 0.001 or lung_ratio > 0.98:
            # Create overlay even for extreme cases
            if SAVE_MASKS:
                img_height, img_width = image_array.shape[:2]
                resize_bounds = (0, 0, img_height, img_width)  # Use full image bounds
                overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
                overlay = create_v13_overlay_separate(image_array, left_binary_mask, right_binary_mask, combined_mask, resize_bounds)
                cv2.imwrite(overlay_path, overlay)
            return image_array
        
        coords = np.column_stack(np.where(combined_mask > 0))
        if len(coords) == 0:
            # Create overlay even when no coords found
            if SAVE_MASKS:
                img_height, img_width = image_array.shape[:2]
                resize_bounds = (0, 0, img_height, img_width)
                overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
                overlay = create_v13_overlay_separate(image_array, left_binary_mask, right_binary_mask, combined_mask, resize_bounds)
                cv2.imwrite(overlay_path, overlay)
            return image_array
        
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)
        
        img_height, img_width = image_array.shape[:2]
        
        # Generous padding around lungs
        generous_padding = 180
        y_min_padded = max(0, y_min - generous_padding)
        x_min_padded = max(0, x_min - generous_padding)
        y_max_padded = min(img_height, y_max + generous_padding)
        x_max_padded = min(img_width, x_max + generous_padding)
        
        lung_center_y = (y_min_padded + y_max_padded) // 2
        lung_center_x = (x_min_padded + x_max_padded) // 2
        
        target_aspect_ratio = TARGET_SIZE[0] / TARGET_SIZE[1]
        lung_width = x_max_padded - x_min_padded
        lung_height = y_max_padded - y_min_padded
        
        min_width = lung_width + 80
        min_height = lung_height + 80
        
        if min_width / min_height > target_aspect_ratio:
            resize_width = min_width
            resize_height = int(resize_width / target_aspect_ratio)
        else:
            resize_height = min_height
            resize_width = int(resize_height * target_aspect_ratio)
        
        resize_x_min = max(0, lung_center_x - resize_width // 2)
        resize_y_min = max(0, lung_center_y - resize_height // 2)
        resize_x_max = min(img_width, resize_x_min + resize_width)
        resize_y_max = min(img_height, resize_y_min + resize_height)
        
        if resize_x_max == img_width:
            resize_x_min = img_width - resize_width
        if resize_y_max == img_height:
            resize_y_min = img_height - resize_height
            
        resize_x_min = max(0, resize_x_min)
        resize_y_min = max(0, resize_y_min)
        
        # Ensure proper separation between green brackets and cyan rectangle
        lung_coords = np.column_stack(np.where(combined_mask > 0))
        if len(lung_coords) > 0:
            actual_y_min, actual_x_min = lung_coords.min(axis=0)
            actual_y_max, actual_x_max = lung_coords.max(axis=0)
            
            min_separation = 100
            top_sep = actual_y_min - resize_y_min
            left_sep = actual_x_min - resize_x_min
            bottom_sep = resize_y_max - actual_y_max
            right_sep = resize_x_max - actual_x_max
            
            if (top_sep < min_separation or left_sep < min_separation or 
                bottom_sep < min_separation or right_sep < min_separation):
                
                expand_amount = min_separation + 50
                resize_x_min = max(0, actual_x_min - expand_amount)
                resize_y_min = max(0, actual_y_min - expand_amount)
                resize_x_max = min(img_width, actual_x_max + expand_amount)
                resize_y_max = min(img_height, actual_y_max + expand_amount)
        
        # Crop to clear area
        if len(image_array.shape) == 3:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max, :]
        else:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max]
        
        # Save overlay with proper crop bounds
        if SAVE_MASKS:
            overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
            resize_bounds = (resize_y_min, resize_x_min, resize_y_max, resize_x_max)
            overlay = create_v13_overlay_separate(image_array, left_binary_mask, right_binary_mask, combined_mask, resize_bounds)
            cv2.imwrite(overlay_path, overlay)
        
        return cropped
        
    except Exception:
        return image_array

# Update the main segment_lungs function to use separate segmentation
def segment_lungs(image):
    """Updated to use separate lung segmentation and return combined mask for compatibility"""
    left_mask, right_mask, combined_mask = segment_lungs_separate(image)
    return combined_mask  # Return combined for backward compatibility


In [ ]:
def segment_lungs(image):
    if model_type == 'torchxray' and segmentation_model is not None:
        return segment_with_torchxray(image)
    else:
        return segment_with_fallback(image)

def enhance_image_preprocessing(image):
    """Enhanced preprocessing for better segmentation detection"""
    if len(image.shape) == 3:
        image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        image_gray = image.copy()
    
    # Histogram equalization for better contrast
    if ENABLE_HISTOGRAM_EQUALIZATION:
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        image_gray = clahe.apply(image_gray)
    
    # Gaussian blur to reduce noise
    if ENABLE_GAUSSIAN_BLUR:
        image_gray = cv2.GaussianBlur(image_gray, (3, 3), 0)
    
    return image_gray

def segment_with_torchxray(image):
    try:
        # Enhanced preprocessing
        image_preprocessed = enhance_image_preprocessing(image)
        
        image_norm = xrv.datasets.normalize(image_preprocessed, 255)
        image_norm = image_norm[None, ...]
        
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        with torch.no_grad():
            output = segmentation_model(image_tensor)
        
        lung_targets = ['Left Lung', 'Right Lung']
        lung_mask = np.zeros((512, 512))
        
        for i, target in enumerate(segmentation_model.targets):
            if target in lung_targets:
                lung_mask += output[0, i].cpu().numpy()
        
        # Debug output
        if ENABLE_DEBUG_OUTPUT:
            print(f"Raw mask stats - Min: {lung_mask.min():.6f}, Max: {lung_mask.max():.6f}, Mean: {lung_mask.mean():.6f}")
        
        lung_mask = cv2.resize(lung_mask, (image.shape[1], image.shape[0]))
        
        # Multiple threshold strategy
        if USE_MULTIPLE_THRESHOLDS:
            thresholds = [MODEL_SENSITIVITY, MODEL_SENSITIVITY * 0.5, MODEL_SENSITIVITY * 0.1, 0.0001]
            best_mask = None
            best_ratio = 0
            
            for threshold in thresholds:
                binary_mask = (lung_mask > threshold).astype(np.uint8)
                lung_ratio = np.sum(binary_mask) / (binary_mask.shape[0] * binary_mask.shape[1])
                
                if ENABLE_DEBUG_OUTPUT:
                    print(f"Threshold {threshold:.6f}: Lung ratio = {lung_ratio:.4f}")
                
                # Target reasonable lung area (2%-40% of image)
                if 0.02 <= lung_ratio <= 0.40:
                    best_mask = binary_mask
                    best_ratio = lung_ratio
                    break
                elif lung_ratio > 0.001 and best_mask is None:  # Accept very small areas as fallback
                    best_mask = binary_mask
                    best_ratio = lung_ratio
            
            if best_mask is not None:
                binary_mask = best_mask
                if ENABLE_DEBUG_OUTPUT:
                    print(f"Selected mask with ratio: {best_ratio:.4f}")
            else:
                binary_mask = (lung_mask > 0.0001).astype(np.uint8)  # Ultra-low fallback
                if ENABLE_DEBUG_OUTPUT:
                    print("Using ultra-low threshold fallback")
        else:
            binary_mask = (lung_mask > MODEL_SENSITIVITY).astype(np.uint8)
        
        # Aggressive morphological operations
        if AGGRESSIVE_MORPHOLOGY:
            kernel_size = 15 if np.sum(binary_mask) > 1000 else 20
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
            
            # Fill holes
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.fillPoly(binary_mask, contours, 1)
        else:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
        
        return binary_mask
        
    except Exception as e:
        if ENABLE_DEBUG_OUTPUT:
            print(f"TorchXRay segmentation failed: {e}")
        return segment_with_fallback(image)

def segment_with_fallback(image):
    # Enhanced fallback with same preprocessing
    image_preprocessed = enhance_image_preprocessing(image)
    
    _, otsu_mask = cv2.threshold(image_preprocessed, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # More aggressive morphology for fallback
    kernel_size = 25 if AGGRESSIVE_MORPHOLOGY else 20
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask_clean = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_OPEN, kernel)
    
    return (mask_clean > 0).astype(np.uint8)


In [ ]:
def create_v13_overlay(image, binary_mask, resize_crop_bounds):
    overlay = image.copy()
    if len(overlay.shape) == 2:
        overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
    
    resize_y_min, resize_x_min, resize_y_max, resize_x_max = resize_crop_bounds
    img_height, img_width = overlay.shape[:2]
    
    # Darken areas outside the resize crop
    outside_resize_mask = np.ones((img_height, img_width), dtype=bool)
    outside_resize_mask[resize_y_min:resize_y_max, resize_x_min:resize_x_max] = False
    overlay[outside_resize_mask] = (overlay[outside_resize_mask] * 0.5).astype(np.uint8)
    
    # Red lung visualization
    lung_areas = binary_mask > 0
    if np.any(lung_areas):
        # Red fill
        lung_fill_colored = np.zeros_like(overlay)
        lung_fill_colored[lung_areas] = [0, 0, 255]
        fill_opacity = max(LUNG_FILL_OPACITY, 0.4)
        overlay[lung_areas] = cv2.addWeighted(
            overlay[lung_areas], 1.0 - fill_opacity, 
            lung_fill_colored[lung_areas], fill_opacity, 0
        )
        
        # Red border
        lung_mask_uint8 = (binary_mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(lung_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        border_mask = np.zeros_like(binary_mask, dtype=np.uint8)
        cv2.drawContours(border_mask, contours, -1, 1, thickness=6)
        
        border_areas = border_mask > 0
        if np.any(border_areas):
            lung_border_colored = np.zeros_like(overlay)
            lung_border_colored[border_areas] = [0, 0, 255]
            border_opacity = max(LUNG_BORDER_OPACITY, 0.6)
            overlay[border_areas] = cv2.addWeighted(
                overlay[border_areas], 1.0 - border_opacity, 
                lung_border_colored[border_areas], border_opacity, 0
            )
    
    # Green corner brackets
    def draw_corner_brackets(img, x1, y1, x2, y2, color, thickness=5, length=60):
        img_height, img_width = img.shape[:2]
        x1 = max(0, min(x1, img_width - 1))
        y1 = max(0, min(y1, img_height - 1))
        x2 = max(0, min(x2, img_width - 1))
        y2 = max(0, min(y2, img_height - 1))
        
        max_length_x = min(length, (x2 - x1) // 4)
        max_length_y = min(length, (y2 - y1) // 4)
        length = max(20, min(max_length_x, max_length_y))
        
        # Top-left
        cv2.line(img, (x1, y1), (min(x1 + length, img_width-1), y1), color, thickness)
        cv2.line(img, (x1, y1), (x1, min(y1 + length, img_height-1)), color, thickness)
        # Top-right
        cv2.line(img, (max(x2 - length, 0), y1), (x2, y1), color, thickness)
        cv2.line(img, (x2, y1), (x2, min(y1 + length, img_height-1)), color, thickness)
        # Bottom-left
        cv2.line(img, (x1, max(y2 - length, 0)), (x1, y2), color, thickness)
        cv2.line(img, (x1, y2), (min(x1 + length, img_width-1), y2), color, thickness)
        # Bottom-right
        cv2.line(img, (x2, max(y2 - length, 0)), (x2, y2), color, thickness)
        cv2.line(img, (max(x2 - length, 0), y2), (x2, y2), color, thickness)
    
    lung_coords = np.where(binary_mask > 0)
    if len(lung_coords[0]) > 0:
        actual_y_min = np.min(lung_coords[0])
        actual_y_max = np.max(lung_coords[0])  
        actual_x_min = np.min(lung_coords[1])
        actual_x_max = np.max(lung_coords[1])
        
        if (actual_x_min >= 0 and actual_y_min >= 0 and 
            actual_x_max < overlay.shape[1] and actual_y_max < overlay.shape[0] and
            actual_x_max > actual_x_min and actual_y_max > actual_y_min):
            draw_corner_brackets(overlay, actual_x_min, actual_y_min, actual_x_max, actual_y_max, (0, 255, 0), 5, 60)
    
    # Cyan rectangle
    cv2.rectangle(overlay, (resize_x_min, resize_y_min), (resize_x_max, resize_y_max), (255, 255, 0), 1)
    
    # Legend
    legend_height = 120
    legend_width = min(700, img_width - 20)
    legend_y_start = img_height - legend_height - 10
    
    legend_background = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
    legend_area = overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width]
    overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width] = cv2.addWeighted(
        legend_area, 0.25, legend_background, 0.75, 0
    )
    
    text_y = legend_y_start + 20
    cv2.putText(overlay, f"RED = Lung segmentation (Fill: {int(LUNG_FILL_OPACITY*100)}%, Border: {int(LUNG_BORDER_OPACITY*100)}%)", 
               (20, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 180), 2)
    cv2.putText(overlay, "GREEN = Segmentation corner brackets", 
               (20, text_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(overlay, f"CYAN = Final resize crop {TARGET_SIZE[0]}x{TARGET_SIZE[1]}", 
               (20, text_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
    
    # Enhanced detection features
    features = []
    if ENABLE_HISTOGRAM_EQUALIZATION: features.append("HistEq")
    if ENABLE_GAUSSIAN_BLUR: features.append("Blur")
    if USE_MULTIPLE_THRESHOLDS: features.append("MultiThresh")
    if AGGRESSIVE_MORPHOLOGY: features.append("AggroMorph")
    features_text = "+".join(features) if features else "Basic"
    
    cv2.putText(overlay, f"Sensitivity: {MODEL_SENSITIVITY} | Enhanced: {features_text}", 
               (20, text_y + 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay, f"Model: {model_type}", 
               (20, text_y + 95), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return overlay

def process_image_with_segmentation(image_array, file_id):
    try:
        binary_mask = segment_lungs(image_array)
        
        # Always save masks and overlays, even for small segmented areas
        if SAVE_MASKS:
            os.makedirs(MASKS_PATH, exist_ok=True)
            
            mask_path = os.path.join(MASKS_PATH, f"{file_id}_mask.png")
            mask_image = (binary_mask * 255).astype(np.uint8)
            cv2.imwrite(mask_path, mask_image)
        
        total_pixels = binary_mask.shape[0] * binary_mask.shape[1]
        lung_pixels = np.sum(binary_mask)
        lung_ratio = lung_pixels / total_pixels
        
        # Only skip cropping for very extreme cases (but still save masks/overlays above)
        if lung_ratio < 0.001 or lung_ratio > 0.98:
            # Create overlay even for extreme cases
            if SAVE_MASKS:
                img_height, img_width = image_array.shape[:2]
                resize_bounds = (0, 0, img_height, img_width)  # Use full image bounds
                overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
                overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
                cv2.imwrite(overlay_path, overlay)
            return image_array
        
        coords = np.column_stack(np.where(binary_mask > 0))
        if len(coords) == 0:
            # Create overlay even when no coords found
            if SAVE_MASKS:
                img_height, img_width = image_array.shape[:2]
                resize_bounds = (0, 0, img_height, img_width)
                overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
                overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
                cv2.imwrite(overlay_path, overlay)
            return image_array
        
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)
        
        img_height, img_width = image_array.shape[:2]
        
        # Generous padding around lungs
        generous_padding = 180
        y_min_padded = max(0, y_min - generous_padding)
        x_min_padded = max(0, x_min - generous_padding)
        y_max_padded = min(img_height, y_max + generous_padding)
        x_max_padded = min(img_width, x_max + generous_padding)
        
        lung_center_y = (y_min_padded + y_max_padded) // 2
        lung_center_x = (x_min_padded + x_max_padded) // 2
        
        target_aspect_ratio = TARGET_SIZE[0] / TARGET_SIZE[1]
        lung_width = x_max_padded - x_min_padded
        lung_height = y_max_padded - y_min_padded
        
        min_width = lung_width + 80
        min_height = lung_height + 80
        
        if min_width / min_height > target_aspect_ratio:
            resize_width = min_width
            resize_height = int(resize_width / target_aspect_ratio)
        else:
            resize_height = min_height
            resize_width = int(resize_height * target_aspect_ratio)
        
        resize_x_min = max(0, lung_center_x - resize_width // 2)
        resize_y_min = max(0, lung_center_y - resize_height // 2)
        resize_x_max = min(img_width, resize_x_min + resize_width)
        resize_y_max = min(img_height, resize_y_min + resize_height)
        
        if resize_x_max == img_width:
            resize_x_min = img_width - resize_width
        if resize_y_max == img_height:
            resize_y_min = img_height - resize_height
            
        resize_x_min = max(0, resize_x_min)
        resize_y_min = max(0, resize_y_min)
        
        # Ensure proper separation between green brackets and cyan rectangle
        lung_coords = np.column_stack(np.where(binary_mask > 0))
        if len(lung_coords) > 0:
            actual_y_min, actual_x_min = lung_coords.min(axis=0)
            actual_y_max, actual_x_max = lung_coords.max(axis=0)
            
            min_separation = 100
            top_sep = actual_y_min - resize_y_min
            left_sep = actual_x_min - resize_x_min
            bottom_sep = resize_y_max - actual_y_max
            right_sep = resize_x_max - actual_x_max
            
            if (top_sep < min_separation or left_sep < min_separation or 
                bottom_sep < min_separation or right_sep < min_separation):
                
                expand_amount = min_separation + 50
                resize_x_min = max(0, actual_x_min - expand_amount)
                resize_y_min = max(0, actual_y_min - expand_amount)
                resize_x_max = min(img_width, actual_x_max + expand_amount)
                resize_y_max = min(img_height, actual_y_max + expand_amount)
        
        # Crop to clear area
        if len(image_array.shape) == 3:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max, :]
        else:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max]
        
        # Save overlay with proper crop bounds
        if SAVE_MASKS:
            overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
            resize_bounds = (resize_y_min, resize_x_min, resize_y_max, resize_x_max)
            overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
            cv2.imwrite(overlay_path, overlay)
        
        return cropped
        
    except Exception:
        return image_array


In [ ]:
# Download from ArchiMed
try:
    user_info = a3conn.getUserInfos()
    csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
    df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
    
    file_id_column = None
    for col in ['FileID', 'file_id', 'File_ID']:
        if col in df.columns:
            file_id_column = col
            break
    
    if file_id_column is None:
        raise ValueError("FileID column not found")
    
    file_ids = df[file_id_column].dropna().unique()
    downloaded_files = []
    
    # Create progress bar with total count
    pbar = tqdm(total=len(file_ids), desc="Downloading files", unit="file")
    
    for file_id in file_ids:
        file_id_str = str(file_id)
        dicom_file_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
        os.makedirs(DOWNLOAD_PATH, exist_ok=True)
        
        if os.path.exists(dicom_file_path):
            downloaded_files.append(dicom_file_path)
            pbar.update(1)
            continue
        
        try:
            result = a3conn.downloadFile(
                int(file_id_str),
                asStream=False,
                destDir=DOWNLOAD_PATH,
                filename=f"{file_id_str}.dcm",
                inWorklist=False
            )
            
            if result and os.path.exists(dicom_file_path):
                downloaded_files.append(dicom_file_path)
            
            pbar.update(1)
                
        except Exception:
            pbar.update(1)
            pass
    
    pbar.close()
    
except Exception:
    downloaded_files = []


In [ ]:
# 🧪 Test function for single DICOM file processing

def process_single_dicom(dicom_path, output_base_name=None):
    """
    Process a single DICOM file for testing purposes
    
    Args:
        dicom_path (str): Path to the DICOM file
        output_base_name (str): Base name for output files (optional)
    
    Returns:
        bool: True if processing was successful
    """
    if not os.path.exists(dicom_path):
        print(f"❌ DICOM file not found: {dicom_path}")
        return False
    
    try:
        # Use filename as base name if not provided
        if output_base_name is None:
            output_base_name = os.path.splitext(os.path.basename(dicom_path))[0]
        
        print(f"🔄 Processing: {os.path.basename(dicom_path)}")
        
        # Create output directories
        os.makedirs(IMAGES_PATH, exist_ok=True)
        os.makedirs(MASKS_PATH, exist_ok=True)
        
        # Output paths
        output_image_path = os.path.join(IMAGES_PATH, f"{output_base_name}.png")
        
        # Process the DICOM file
        success = convert_dicom_to_image(dicom_path, output_image_path)
        
        if success:
            print(f"✅ Successfully processed {output_base_name}")
            print(f"📁 Check output files:")
            print(f"   • Image: {output_image_path}")
            
            if SAVE_MASKS:
                expected_masks = [
                    f"{output_base_name}_left_lung_mask.png",
                    f"{output_base_name}_right_lung_mask.png", 
                    f"{output_base_name}_combined_mask.png",
                    f"{output_base_name}_overlay.png"
                ]
                
                for mask_file in expected_masks:
                    mask_path = os.path.join(MASKS_PATH, mask_file)
                    if os.path.exists(mask_path):
                        print(f"   • {mask_file} ✅")
                    else:
                        print(f"   • {mask_file} ❌")
            
            return True
        else:
            print(f"❌ Failed to process {output_base_name}")
            return False
            
    except Exception as e:
        print(f"❌ Error processing {dicom_path}: {e}")
        return False

# Example usage:
# process_single_dicom("/path/to/your/file.dcm", "test_image")


In [ ]:
# Convert DICOM to images
def convert_dicom_to_image(dicom_path, output_path, target_size=TARGET_SIZE):
    try:
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        
        dicom_data = pydicom.dcmread(dicom_path)
        image_array = dicom_data.pixel_array
        
        if hasattr(dicom_data, 'PhotometricInterpretation'):
            if dicom_data.PhotometricInterpretation == 'MONOCHROME1':
                image_array = np.max(image_array) - image_array
        
        if image_array.max() > 255:
            image_array = ((image_array - image_array.min()) / 
                          (image_array.max() - image_array.min()) * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)
        
        processed_image = process_image_with_segmentation_separate(image_array, file_id)
        
        if len(processed_image.shape) == 2:
            pil_image = Image.fromarray(processed_image, mode='L')
        else:
            pil_image = Image.fromarray(processed_image)
        
        current_width, current_height = pil_image.size
        target_width, target_height = target_size
        current_ratio = current_width / current_height
        target_ratio = target_width / target_height
        
        if current_ratio > target_ratio:
            new_width = int(current_height * target_ratio)
            new_height = current_height
            left = (current_width - new_width) // 2
            top = 0
            right = left + new_width
            bottom = current_height
        else:
            new_width = current_width
            new_height = int(current_width / target_ratio)
            left = 0
            top = (current_height - new_height) // 2
            right = current_width
            bottom = top + new_height
        
        pil_image = pil_image.crop((left, top, right, bottom))
        pil_image = pil_image.resize(target_size, Image.Resampling.LANCZOS)
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        pil_image.save(output_path)
        return True
        
    except Exception:
        return False

# Convert files
if CONVERT and downloaded_files:
    os.makedirs(IMAGES_PATH, exist_ok=True)
    converted_count = 0
    
    with tqdm(total=len(downloaded_files), desc="Converting DICOM files", unit="file") as pbar:
        for dicom_path in downloaded_files:
            file_id = os.path.splitext(os.path.basename(dicom_path))[0]
            output_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
            
            if convert_dicom_to_image(dicom_path, output_path):
                converted_count += 1
            pbar.update(1)
            pbar.set_postfix({"Converted": converted_count})


In [ ]:
# 🚀 Execute the complete pipeline

print("="*60)
print("🫁 ArchiMed Images V1.4 - Separate Lung Detection Pipeline")
print("="*60)

# Step 1: Connect to ArchiMed and get file list
available_files = []
authenticated = False

if archimed_available:
    print("\n🔗 Step 1: ArchiMed already connected")
    try:
        # ArchiMed connection was established in imports cell
        authenticated = True
        
        # Load CSV file to get file IDs
        csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
        if not os.path.exists(csv_path):
            print(f"❌ CSV file not found: {csv_path}")
            print("📂 Please check the CSV_FOLDER and CSV_LABELS_FILE settings")
        else:
            df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
            
            # Find the FileID column
            file_id_column = None
            for col in ['FileID', 'file_id', 'File_ID']:
                if col in df.columns:
                    file_id_column = col
                    break
            
            if file_id_column is None:
                print(f"❌ FileID column not found in CSV. Available columns: {list(df.columns)}")
            else:
                file_ids = df[file_id_column].dropna().unique()
                print(f"📋 Found {len(file_ids)} files in CSV: {CSV_LABELS_FILE}")
                available_files = [str(fid) for fid in file_ids]
        
    except Exception as e:
        print(f"❌ ArchiMed connection failed: {e}")
        print("💡 Tip: Check your ArchiMed credentials and network connection")
        authenticated = False
else:
    print("\n⚠️ ArchiMed not available - cannot search for files")

# Step 2: Check local files and download missing ones
print(f"\n📁 Step 2: Checking local files and downloading if needed...")

# Create download directory
os.makedirs(DOWNLOAD_PATH, exist_ok=True)

dicom_files = []
files_to_download = []
local_files_found = 0

if available_files:
    print(f"🔍 Checking {len(available_files)} files...")
    
    for file_id in available_files:
        dicom_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
        
        if os.path.exists(dicom_path):
            dicom_files.append(dicom_path)
            local_files_found += 1
        else:
            if authenticated and DOWNLOAD_IF_MISSING:
                files_to_download.append(file_id)
    
    print(f"✅ Found {local_files_found} files locally")
    print(f"📥 Need to download {len(files_to_download)} files")
    
    # Download missing files
    if files_to_download and authenticated:
        print(f"🌐 Downloading {len(files_to_download)} missing files...")
        
        download_success = 0
        pbar = tqdm(total=len(files_to_download), desc="Downloading", unit="file")
        
        for file_id in files_to_download:
            try:
                dicom_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
                
                result = a3conn.downloadFile(
                    int(file_id),
                    asStream=False,
                    destDir=DOWNLOAD_PATH,
                    filename=f"{file_id}.dcm",
                    inWorklist=False
                )
                
                if result and os.path.exists(dicom_path):
                    dicom_files.append(dicom_path)
                    download_success += 1
                
            except Exception as download_error:
                print(f"⚠️ Failed to download {file_id}: {download_error}")
            
            pbar.update(1)
        
        pbar.close()
        print(f"✅ Successfully downloaded {download_success}/{len(files_to_download)} files")
    
    elif files_to_download and not authenticated:
        print(f"❌ Cannot download {len(files_to_download)} missing files - ArchiMed not authenticated")
    
else:
    # No file list available, scan local directory
    print("📂 No file list available, scanning local directory...")
    local_patterns = [
        os.path.join(DOWNLOAD_PATH, "*.dcm"),
        os.path.join(DOWNLOAD_PATH, "*.DCM"),
        os.path.join(DOWNLOAD_PATH, "**/*.dcm"),
        os.path.join(DOWNLOAD_PATH, "**/*.DCM"),
    ]
    
    for pattern in local_patterns:
        dicom_files.extend(glob.glob(pattern, recursive=True))
    
    dicom_files = list(set(dicom_files))  # Remove duplicates
    print(f"📁 Found {len(dicom_files)} local DICOM files")

print(f"🎯 Total DICOM files ready for processing: {len(dicom_files)}")

# Step 3: Convert DICOM to images with separate lung segmentation
print(f"\n🔄 Step 3: Converting DICOM files with separate lung detection...")

if CONVERT and dicom_files:
    os.makedirs(IMAGES_PATH, exist_ok=True)
    converted_count = 0
    
    print(f"Processing {len(dicom_files)} DICOM files...")
    print(f"Output paths:")
    print(f"  📁 Images: {IMAGES_PATH}")
    print(f"  📁 Masks: {MASKS_PATH}")
    
    with tqdm(total=len(dicom_files), desc="Converting DICOM files", unit="file") as pbar:
        for dicom_path in dicom_files:
            file_id = os.path.splitext(os.path.basename(dicom_path))[0]
            output_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
            
            if convert_dicom_to_image(dicom_path, output_path):
                converted_count += 1
            pbar.update(1)
            pbar.set_postfix({"Converted": converted_count})
    
    print(f"✅ Successfully converted {converted_count} images")
    
    # Summary of outputs
    print(f"\n📊 Processing Summary:")
    print(f"  • Converted images: {converted_count}")
    print(f"  • Model type: {model_type}")
    print(f"  • Sensitivity: {MODEL_SENSITIVITY}")
    
    if SAVE_MASKS:
        print(f"\n📁 Output files for each image:")
        print(f"  • {IMAGES_PATH}/{{file_id}}.png - Processed image")
        print(f"  • {MASKS_PATH}/{{file_id}}_left_lung_mask.png - Left lung mask")
        print(f"  • {MASKS_PATH}/{{file_id}}_right_lung_mask.png - Right lung mask")
        print(f"  • {MASKS_PATH}/{{file_id}}_combined_mask.png - Combined lung mask")
        print(f"  • {MASKS_PATH}/{{file_id}}_overlay.png - Color-coded visualization")
        
        # Check if masks were created
        mask_files = []
        if os.path.exists(MASKS_PATH):
            mask_files = [f for f in os.listdir(MASKS_PATH) if f.endswith('_mask.png')]
        
        print(f"\n🎯 Created {len(mask_files)} mask files")
        
        if mask_files:
            print(f"✅ Pipeline completed successfully!")
            print(f"🔍 Check {MASKS_PATH} for visualization overlays")
        else:
            print(f"⚠️  No mask files found - check segmentation settings")
    
else:
    print(f"❌ No files to convert or CONVERT=False")

print(f"\n{'='*60}")
print(f"🏁 Pipeline execution completed!")
print(f"{'='*60}")


In [ ]:
# 🚀 Quick Test: Process just one DICOM file to test ArchiMed connection

def process_single_dicom():
    """Quick test function to process just one DICOM file"""
    print("="*60)
    print("🧪 Quick Test: Processing Single DICOM File")
    print("="*60)
    
    # Step 1: ArchiMed connection already established
    if archimed_available:
        print("\n🔗 ArchiMed connection available")
        print(f"✅ Using established connection from imports cell")
    
    # Step 2: Find one DICOM file to test
    print(f"\n📁 Looking for test DICOM file...")
    test_file = None
    
    local_patterns = [
        os.path.join(DOWNLOAD_PATH, "*.dcm"),
        os.path.join(DOWNLOAD_PATH, "*.DCM"),
    ]
    
    for pattern in local_patterns:
        files = glob.glob(pattern)
        if files:
            test_file = files[0]  # Take first file
            break
    
    if test_file is None:
        print(f"❌ No DICOM files found in {DOWNLOAD_PATH}")
        return
    
    print(f"🎯 Test file: {os.path.basename(test_file)}")
    
    # Step 3: Process the test file
    print(f"\n🔄 Processing test file...")
    file_id = os.path.splitext(os.path.basename(test_file))[0]
    output_path = os.path.join(IMAGES_PATH, f"{file_id}_test.png")
    
    os.makedirs(IMAGES_PATH, exist_ok=True)
    
    try:
        if convert_dicom_to_image(test_file, output_path):
            print(f"✅ Test conversion successful!")
            print(f"📁 Output: {output_path}")
            
            if SAVE_MASKS:
                mask_files = [
                    f"{file_id}_left_lung_mask.png",
                    f"{file_id}_right_lung_mask.png", 
                    f"{file_id}_combined_mask.png",
                    f"{file_id}_overlay.png"
                ]
                
                existing_masks = []
                for mask_file in mask_files:
                    mask_path = os.path.join(MASKS_PATH, mask_file)
                    if os.path.exists(mask_path):
                        existing_masks.append(mask_file)
                
                print(f"🎯 Created masks: {len(existing_masks)}/{len(mask_files)}")
                for mask in existing_masks:
                    print(f"  ✅ {mask}")
        else:
            print(f"❌ Test conversion failed")
            
    except Exception as e:
        print(f"❌ Test processing failed: {e}")
    
    print("="*60)
    print("🏁 Quick test completed!")
    print("="*60)

# Run the quick test
process_single_dicom()
